In [ ]:
from dotenv import load_dotenv
from helpers.utils import Utils

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.patches import Rectangle
from ipywidgets import interactive, VBox, Layout, HBox, IntSlider, FloatSlider, IntRangeSlider

from models.peer import Peer

%config InlineBackend.figure_format = 'retina'

In [ ]:
use_snapshot = True
min_version = "2.0.7"
budget_dollar = 7500
token_price = 0.05
min_stake_wxhopr = 10e3
thrs_stake_wxhopr = 75e3

### Get data


In [ ]:
if use_snapshot or not load_dotenv():
    # change the following line to use a different snapshot (see `snapshot` folder)
    peers, safes, topology = Utils.loadSnapshot("2024_02_22") 
else:
    safes = await Utils.getSafesData()
    topology = await Utils.getTopologyData()
    peers = await Utils.getPeers()

    Utils.dumpSnapshot(safes, topology, peers)

print(f"Subgraph size: {len(safes)}")
print(f"Topology size: {len(topology)}" )
print(f"Number of peers: {len(peers)}")
    

### Define and apply economic model

In [ ]:
def plot_hist(ax: plt.Axes, peers: list[Peer], bin: list[int], limits: list[int], token_price: float, watermark: str):
    # data
    stakes = [peer.split_stake for peer in peers]
    rewards = [peer.rewards for peer in peers]

    apr = sum(rewards) / sum(stakes) * 12 if sum(stakes) > 0 else 0
  
    texts = [
        f"Stakes: {sum(stakes):_.0f} wxHOPR",
        f"Rewards: {sum(rewards):_.0f} wxHOPR -> Budget: {sum(rewards)*token_price:_.0f} $",
        f"APR (avg.): {apr:.2%}",
        f"Peers: {len(peers)}"
    ]


    # plot
    counts, _, bars = ax.hist(stakes, bins=bin, facecolor="#000050", edgecolor='black', linewidth=1.2, rwidth=.85)
    max_height = max(round(counts.max() / 20 + .5) * 20, 1)
    areas = [
        (((0, 0), limits[0], max_height), "#ff0000"),
        (((limits[0], 0), limits[1]-limits[0], max_height), "#ffffa0")
    ]

    # settings
    ax.bar_label(bars)
    ax.set_xlim(bin[0], bin[-1])
    ax.set_ylim(0, max_height)
    ax.set_xlabel("Stake (wxHOPR)")
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    [ax.add_patch(Rectangle(*loc, color=color, alpha=.4, zorder=0)) for loc, color in areas]

    for idx, text in enumerate(texts):
        test_item = ax.text(.95, .95-idx*.1, text, transform=ax.transAxes, ha='right', va='top')
        test_item.set_bbox(dict(facecolor='#b4f0ff', alpha=.5, edgecolor='white'))

    ax.text(0.5, 0.5, watermark, transform=ax.transAxes,
        fontsize=40, color='gray', alpha=0.5,
        ha='center', va='center', rotation=30)

def plot_multiple_hist(bin_count: int, limits: list[int], token_price: float, target_apr: float, random_extra_stake_range: list[int], num_extra_nodes: int, extra_node_apr_factor: float, overlay: str = ""):
    eligibles = Utils.getEligiblesPeers(topology, peers, safes, min_version)

    ###############################################################################
    for _ in range(num_extra_nodes):
        eligibles.append(Peer.extra(random_extra_stake_range))

    total_stake = sum(peer.split_stake for peer in eligibles if peer.split_stake > limits[0])
    budget_dollars = target_apr * total_stake / 12 / 100 * token_price

    eligibles = Utils.getRewardProbability(eligibles, token_price, budget_dollars, limits)

    # probability adjustment
    for peer in eligibles:
        if "extra_address" not in peer.address.address:
            continue
        peer.reward_probability *= extra_node_apr_factor

    total_probability = sum(peer.reward_probability for peer in eligibles)

    # normalize the probability
    for peer in eligibles:
        peer.reward_probability /= total_probability
    ###############################################################################

        
    separators = [limits[1], 1000e3]

    splits = [[] for _ in separators]
    for peer in eligibles:
        for idx, sep in enumerate(separators):
            if peer.split_stake < sep:
                splits[idx].append(peer)
                break
        
    bins = []
    low = 0
    for high in separators:
        bins.append(Utils.binsFromRange(low, high, bin_count))
        low = high

    _, axes = plt.subplots(1, len(splits), figsize=(8*len(splits), 4.5), sharex=False, sharey=False)
    axes = axes.flatten() if not isinstance(axes, plt.Axes) else [axes]

    for idx, (ax, split, bin) in enumerate(zip(axes, splits, bins)):
        plot_hist(ax, split, bin, limits, token_price, overlay)

    _ = axes[0].set_ylabel("Number of peers")
    plt.subplots_adjust(wspace=0.06)

def plot_multiple_hist_interactive():
    style = {'description_width': 'initial', 'value_width': 'initial'}

    # histogram controls
    bin_slider = IntSlider(min=5, max=30, step=1, value=20, description="Bins count", style=style)

    # distribution controls
    limits_slider = IntRangeSlider(min=0, max=500e3, step=5e3, description="Eligibility thres.", value=[min_stake_wxhopr, thrs_stake_wxhopr], style=style, layout={"width": "600px"})
    budget_dollars_slider = IntSlider(min=0, max=20000, step=500, value=budget_dollar, description="Budget ($)", style=style)
    apr_slider = FloatSlider(min=0.1, max=20.0, step=0.1, value=15, description="Target APR", style=style)
    token_price_slider = FloatSlider(min=0.01, max=0.20, step=0.001, value=token_price, description="Token price ($)",readout_format='.3f', style=style)

    # extra investors sliders
    random_extra_stake_slider = IntRangeSlider(min=0, max=800e3, step=25e3, description="Random stake", value=[250e3, 350e3], style=style, layout={"width": "600px"})
    num_extra_nodes_slider = IntSlider(min=0, max=300, step=1, value=0, description="Extra nodes", style=style)
    extra_node_apr_factor_slider = FloatSlider(min=1, max=10.0, step=0.1, value=1, description="Extra APR boost", style=style)

    widgets = interactive(plot_multiple_hist, limits=limits_slider, bin_count=bin_slider, token_price=token_price_slider, target_apr=apr_slider, random_extra_stake_range=random_extra_stake_slider, num_extra_nodes=num_extra_nodes_slider,extra_node_apr_factor=extra_node_apr_factor_slider, overlay="Simulation")

    reward_controls = HBox(widgets.children[1:4], layout = Layout(flex_flow='row wrap'))
    extra_investor_controls = HBox(widgets.children[4:7], layout = Layout(flex_flow='row wrap'))
    hist_controls = widgets.children[0]

    plot_multiple_hist(limits=[min_stake_wxhopr, thrs_stake_wxhopr], bin_count=20,token_price=token_price, target_apr=15.0, random_extra_stake_range=[0], num_extra_nodes=0, extra_node_apr_factor=1, overlay="Status quo")

    display(VBox([reward_controls, extra_investor_controls, hist_controls]))
    display(widgets.children[-1])

In [ ]:
plot_multiple_hist_interactive()